## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [9]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [10]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [12]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = "sqlite:///backend.db"
client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)
experiments = client.search_experiments()
for experiment in experiments:
    print(experiment)

<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1717082667339, experiment_id='0', last_update_time=1717082667339, lifecycle_stage='active', name='Default', tags={}>


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/30 21:01:30 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
/home/ganesh/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


default artifacts URI: '/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/running-mlflows-examples/artifacts_local/1/e47efcc258c24113a5aa8b48def9e68c/artifacts'


In [14]:
experiments = client.search_experiments()
for experiment in experiments:
    print(experiment)

<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1717082667339, experiment_id='0', last_update_time=1717082667339, lifecycle_stage='active', name='Default', tags={}>


### Interacting with the model registry

In [15]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [16]:
experiments = client.search_experiments()
for experiment in experiments:
    print(experiment)

<Experiment: artifact_location='/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/running-mlflows-examples/artifacts_local/1', creation_time=1717083090878, experiment_id='1', last_update_time=1717083090878, lifecycle_stage='active', name='my-experiment-1', tags={}>
<Experiment: artifact_location='/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/running-mlflows-examples/artifacts_local/0', creation_time=1717083010817, experiment_id='0', last_update_time=1717083010817, lifecycle_stage='active', name='Default', tags={}>


In [17]:
#list registed model 
models = mlflow.search_model_versions()

# Display the registered models
print(models)

[]


In [27]:
runs = client.search_runs(
    experiment_ids="1"
)
for run in runs:
    print(f"run id : {run.info.run_id}")

run id : e47efcc258c24113a5aa8b48def9e68c


In [28]:
run_id = "e47efcc258c24113a5aa8b48def9e68c"
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/05/30 21:14:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1717083895765, current_stage='None', description='', last_updated_timestamp=1717083895765, name='iris-classifier', run_id='e47efcc258c24113a5aa8b48def9e68c', run_link='', source='/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/running-mlflows-examples/artifacts_local/1/e47efcc258c24113a5aa8b48def9e68c/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>